In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [36]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 50, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(50, 150, 5)
        self.fc1 = nn.Linear(150 * 5 * 5, 300)
        self.fc2 = nn.Linear(300, 200)
        self.fc3 = nn.Linear(200, 10)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

trainloss=[]
trainaccuracy=[]
testloss=[]
testaccuracy=[]

In [37]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0


    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in trainloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 train images: {100 * correct // total} %')   
    trainaccuracy.append(100 * correct // total)

    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    testaccuracy.append(100 * correct // total)

print('Finished Training')

[1,  2000] loss: 2.026
[1,  4000] loss: 1.632
[1,  6000] loss: 1.490
[1,  8000] loss: 1.373
[1, 10000] loss: 1.295
[1, 12000] loss: 1.254
Accuracy of the network on the 10000 train images: 60 %
Accuracy of the network on the 10000 test images: 59 %
[2,  2000] loss: 1.112
[2,  4000] loss: 1.065
[2,  6000] loss: 1.046
[2,  8000] loss: 1.025
[2, 10000] loss: 0.973
[2, 12000] loss: 0.952
Accuracy of the network on the 10000 train images: 70 %
Accuracy of the network on the 10000 test images: 65 %
[3,  2000] loss: 0.806
[3,  4000] loss: 0.819
[3,  6000] loss: 0.817
[3,  8000] loss: 0.809
[3, 10000] loss: 0.791
[3, 12000] loss: 0.787
Accuracy of the network on the 10000 train images: 73 %
Accuracy of the network on the 10000 test images: 67 %
[4,  2000] loss: 0.622
[4,  4000] loss: 0.642
[4,  6000] loss: 0.629
[4,  8000] loss: 0.638
[4, 10000] loss: 0.657
[4, 12000] loss: 0.664
Accuracy of the network on the 10000 train images: 83 %
Accuracy of the network on the 10000 test images: 72 %
[5, 

In [42]:
print('The maximum training accuracy is ',max(trainaccuracy))
print('The maximum testing accuracy is ',max(testaccuracy))

The maximum training accuracy is  98
The maximum testing accuracy is  74
